In [ ]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)
import nglview as nv
import matplotlib.pyplot as plt

# Visualizing the trajectory using nglview
It is always a good idea to render your molecular dynamics simulations to check if anything strange is going on. 

If your simulation has just started the movie will be very short. Re-run the following cell to update results.

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
import MDAnalysis as mda

# The trajectory is imported through MDAnalysis
mda_traj = mda.Universe("./H2O/traj.dump", topology_format="LAMMPSDUMP")

# Atoms are named appropriately
atomtypes_to_names = []
for i in range(len(mda_traj.atoms)):
    if mda_traj.atoms[i].type == "1":
        atomtypes_to_names.append("O")
    elif mda_traj.atoms[i].type == "2":
        atomtypes_to_names.append("H")
mda_traj.add_TopologyAttr("name", atomtypes_to_names)

mda_view = nv.show_mdanalysis(mda_traj)
mda_view.clear_representations()
mda_view.add_representation('spacefill',radius =0.8)
mda_view

#IF RENDER LOOKS TRIPPY, TRY DIFFERENT BROWSER!

# Energy conservation - NVE ensemble
Using a small script provided in the directory we can import all observables saved in the log.lammps file. The  instantaneous temperature in molecular dynamics simulations is defined through: $E_{kin} = \frac{dim}{2} N_{atoms} k_B T$.

The total energy should stay as conserved as possible, only allowing small fluctuations. A drift in energy should not be observed. Is this fulfilled for our simulation? How can we reduce fluctuations?

In [ ]:
import log

logfile = log.File("./H2O/log.lammps")
PotEng = logfile.get('PotEng')
Press = logfile.get('Press')


plt.plot(logfile.get('Temp'))
plt.xlabel('time (fs)')
plt.ylabel('T (K)')
plt.show()

plt.plot(logfile.get('TotEng'))
plt.xlabel('time (fs)')
plt.ylabel('Total Energy (eV)')
plt.show()

plt.plot(logfile.get('Press'))
plt.xlabel('time (fs)')
plt.ylabel('Pressure (bar)')
plt.show()


Since we use the NVE ensemble, temperature and pressure are not controlled. Calculate the mean temperature and pressure and evaluate whether we are close to standard temperature and pressure.

In [ ]:
# COMPLETE MISSING ARGUMENT OF GET()
inport numpy as np
print(np.mean(logfile.get('')))

# Calculation of the O-O pair correlation function
The pair correlation function $ g(r)=\frac{V}{4\pi r^{2}N^{2}} \langle\sum_{i}\sum_{j\neq i} \delta (r-r_{ij}) \rangle$ is the conditional probability density of finding a particle at a radial distance $r$, given that there is a particle at the coordinate origin. It provides a measure of the local spatial ordering in a fluid and is an important material property that we want reproduce with our machine learning force-field. Find how a pair correlation function should look like for fluids and compare it with your results.

If your MD-simulation has less than 2000 steps the function will look a bit rough. Re-run the following cell after some time. It will smooth out over time.

In [ ]:
import MDAnalysis.analysis.rdf

mda_traj = mda.Universe("./H2O/traj.dump", topology_format="LAMMPSDUMP")
Oxygen_atoms = mda_traj.select_atoms("type 2")
rdf = mda.analysis.rdf.InterRDF(Oxygen_atoms, Oxygen_atoms, bins = 100, exclusion_block=(1, 1), range =(0,8))
rdf.run()

plt.plot(rdf.results.bins, rdf.results.rdf)
plt.xlabel('Radius (angstrom)')
plt.ylabel('Radial distribution')
plt.xlim(0,8)

# Determination of diffusion behaviour
Finally, we can also check the diffusion behaviour of our water molecules by looking at the mean square displacement after a time interval $\tau$. For a brownian particle, the mean square displacement in three dimensions will grow linearly with $MSD = 2 D \tau$ where $D$ is its diffusion coefficient. 

Is this fulfilled everywhere? What happens at the "ballistic"-regime?

In [ ]:
import MDAnalysis.analysis.msd as msd
import numpy as np

mda_traj = mda.Universe("./H2O/traj.dump", topology_format="LAMMPSDUMP")

MSD = msd.EinsteinMSD(mda_traj, select='all', msd_type='xyz', fft=False)
MSD.run()
nframes = MSD.n_frames
timestep = 0.0025  # = 5*0.5 fs in picoseconds
lagtimes = np.arange(nframes)*timestep

msd =  MSD.results.timeseries

fig = plt.figure()
ax = plt.axes()
# we plot the actual MSD
ax.plot(lagtimes, msd, color="black", ls="-", label=r'Our simulation')

# we plot an exact result for a Brownian particle with D = 1. We do not have the same D!
exact = lagtimes*6
ax.plot(lagtimes, exact, color="black", ls="--", label=r'$y=2 \tau$')
plt.xlabel('time (ps)')
plt.ylabel('Mean Square Displacement (angstrom$^2$)')
plt.legend()
plt.show()

After the simulation has finished, we can calculate the diffusion coefficient ourselves using linear regression:

In [ ]:
from scipy.stats import linregress
start_index = 100

linear_model = linregress(lagtimes[start_index:],msd[start_index:])
slope = linear_model.slope
error = linear_model.rvalue
# dim_fac is 3 as we computed a 3D msd with 'xyz'
D = slope * 1/(2*MSD.dim_fac)

print(D)

Do you want to improve your results? Start your simulation again but change the number of timesteps to a higher number (run $number_of_timestep). If your CPU has more than one core, you can let it run during work on the next exercises and check later.